In [1]:
from typing import *
import libcst as cst
import networkx as nx
import pandas as pd
import numpy as np

In [2]:
from alkh import cst_utils

In [3]:
file_path = 'play.py'

In [4]:
line_number = 1

In [5]:
file_lines = open(file_path, 'r').readlines()
file_content = open(file_path, 'r').read()

In [ ]:
var_name = cst_utils.get_variable_name(file_lines[line_number - 1])

In [ ]:
file_cst = cst.parse_module(open(file_path, 'r').read())

In [ ]:
file_cst

In [ ]:
class FunctionCollector(cst.CSTVisitor):
    def __init__(self):
        self._names: List[str] = []
    
    def visit_FunctionDef(self, node: cst.FunctionDef) -> None:
#         wrapper = cst.metadata.MetadataWrapper(node)
        collector = AssignCollector()
#         wrapper.visit(collector)
        node.visit(collector)
        self._names.append(collector.names)
        pass

    def leave_FunctionDef(self, node: cst.FunctionDef) -> None:
        pass

In [ ]:
class AssignCollector(cst.CSTVisitor):
#     METADATA_DEPENDENCIES = (cst.metadata.PositionProvider,)
    
    def __init__(self):
        self.names: List[str] = []
    
    def visit_Assign(self, node: cst.FunctionDef) -> None:
#         pos = self.get_metadata(cst.metadata.PositionProvider, node).start
        collector = ValueCollector()
        node.value.visit(collector)
        value_dict = {'names': collector.names, 'ints': collector.ints, 'floats': collector.floats}
#         self.names.append((node.targets[0].target.value, value_dict, pos.line))
        self.names.append((node.targets[0].target.value, value_dict))

In [ ]:
class ValueCollector(cst.CSTVisitor):
    def __init__(self):
        self.names: List[str] = []
        self.ints: List[str] = []
        self.floats: List[str] = []
    
    def visit_Name(self, node: cst.FunctionDef) -> None:
        self.names.append(node.value)
        
    def visit_Integer(self, node: cst.FunctionDef) -> None:
        self.ints.append(node.value)
        
    def visit_Float(self, node: cst.FunctionDef) -> None:
        self.floats.append(node.value)

In [ ]:
visitor = FunctionCollector()
# file_cst.visit(visitor)

In [ ]:
visitor._names

In [ ]:
a = visitor._names[1]
a

In [ ]:
b = pd.DataFrame(a, columns=['assigned', 'data'])
b

In [ ]:
b['assignee'] = b['data'].apply(lambda x: x['names'])

In [ ]:
b

In [ ]:
dg = nx.DiGraph()

In [ ]:
var_names = set(list(b['assignee'].explode().dropna().unique()) + list(b['assigned'].unique()))
var_names

In [ ]:
for name in var_names:
    dg.add_node(name)

In [ ]:
for index, a_series in b.iterrows():
    if a_series['assignee']:
        for assignee in a_series['assignee']:
            dg.add_edge(assignee, a_series['assigned'])

In [ ]:
dg

In [ ]:
nx.draw_networkx(dg)

In [ ]:
ancestors = nx.ancestors(dg, 'd')

In [ ]:
b[b['assigned'].isin(ancestors)]

In [ ]:
print(file_cst.code)

In [ ]:
class NamePrinter(cst.CSTVisitor):
    METADATA_DEPENDENCIES = (cst.metadata.PositionProvider,)

    def visit_Name(self, node: cst.Name) -> None:
        pos = self.get_metadata(cst.metadata.PositionProvider, node).start
        print(f"{node.value} found at line {pos.line}, column {pos.column}")

In [ ]:
wrapper = cst.metadata.MetadataWrapper(cst.parse_module("x = 1"))
result = wrapper.visit(NamePrinter())

In [ ]:
wrapper = cst.metadata.MetadataWrapper(cst.parse_module(open(file_path, 'r').read()))
result = wrapper.visit(NamePrinter())

In [ ]:
k = cst.parse_module(open(file_path, 'r').read())

In [ ]:
# k.body[0]

In [ ]:
wrapper = cst.metadata.MetadataWrapper(k)
# wrapper = cst.metadata.MetadataWrapper(k.body[0].name)
result = wrapper.visit(NamePrinter())

In [ ]:
class FunctionCollector(cst.CSTVisitor):
    METADATA_DEPENDENCIES = (cst.metadata.PositionProvider,)
    def __init__(self):
        self._names: List[str] = []
        self.names: List[str] = []
    
    def visit_FunctionDef(self, node: cst.FunctionDef) -> None:
#         wrapper = cst.metadata.MetadataWrapper(node)
#         pos = self.get_metadata(cst.metadata.PositionProvider, node).start
#         collector = AssignCollector()
#         wrapper.visit(collector)
#         node.visit(collector)
#         self._names.append((collector.names, pos.line))
        pass

    def leave_FunctionDef(self, node: cst.FunctionDef) -> None:
        pass
    
    def visit_Assign(self, node: cst.FunctionDef) -> None:
        pos = self.get_metadata(cst.metadata.PositionProvider, node).start
        collector = ValueCollector()
        node.value.visit(collector)
        value_dict = {'names': collector.names, 'ints': collector.ints, 'floats': collector.floats}
        self.names.append((node.targets[0].target.value, value_dict, pos.line))
#         self.names.append((node.targets[0].target.value, value_dict))

In [ ]:
class AssignCollector(cst.CSTVisitor):
    METADATA_DEPENDENCIES = (cst.metadata.PositionProvider,)
    
    def __init__(self):
        self.names: List[str] = []
    
    def visit_Assign(self, node: cst.FunctionDef) -> None:
        pos = self.get_metadata(cst.metadata.PositionProvider, node).start
        collector = ValueCollector()
        node.value.visit(collector)
        value_dict = {'names': collector.names, 'ints': collector.ints, 'floats': collector.floats}
#         self.names.append((node.targets[0].target.value, value_dict, pos.line))
        self.names.append((node.targets[0].target.value, value_dict))

In [ ]:
visitor = FunctionCollector()
wrapper = cst.metadata.MetadataWrapper(file_cst)
jj = wrapper.visit(visitor)

In [ ]:
visitor.names

In [ ]:
b = pd.DataFrame(visitor.names, columns=['assigned', 'data', 'line'])
b['assignee'] = b['data'].apply(lambda x: x['names'])
b

In [ ]:
var_names = set(list(b['assignee'].explode().dropna().unique()) + list(b['assigned'].unique()))
var_names

In [ ]:
dg = nx.DiGraph()
for name in var_names:
    dg.add_node(name)
for index, a_series in b.iterrows():
    if a_series['assignee']:
        for assignee in a_series['assignee']:
            dg.add_edge(assignee, a_series['assigned'])
nx.draw_networkx(dg)

In [ ]:
var_name = 'll'
ancestors = nx.ancestors(dg, var_name)
c = b[b['assigned'].isin(ancestors.union(set([var_name])))]
c

In [ ]:
lines = open(file_path, 'r').readlines()
print("".join(lines))

In [ ]:
print("".join(np.array(open(file_path, 'r').readlines())[c['line'].values- 1]))

In [ ]:
line_cst = cst.parse_module(lines[5].strip())

In [ ]:
line_cst

In [ ]:
class AssignCollector(cst.CSTVisitor):   
    def __init__(self):
        self.names: List[str] = []
    
    def visit_Assign(self, node: cst.FunctionDef) -> None:
        targets = [target.target.value for target in node.targets]
        self.names.append(targets)

In [ ]:
collector = AssignCollector()
res = line_cst.visit(collector)

In [ ]:
collector.names

In [ ]:
class FunctionCollector(cst.CSTVisitor):
    METADATA_DEPENDENCIES = (cst.metadata.PositionProvider,)
    def __init__(self):
        self._names: List[str] = []
        self.names: List[str] = []
    
    def visit_Assign(self, node: cst.FunctionDef) -> None:
        pos = self.get_metadata(cst.metadata.PositionProvider, node).start
        collector = ValueCollector()
        node.value.visit(collector)
        value_dict = {'names': collector.names, 'ints': collector.ints, 'floats': collector.floats}
        self.names.append((node.targets[0].target.value, value_dict, pos.line))

In [ ]:
class ValueCollector(cst.CSTVisitor):
    def __init__(self):
        self.names: List[str] = []
        self.ints: List[str] = []
        self.floats: List[str] = []
    
    def visit_Name(self, node: cst.FunctionDef) -> None:
        self.names.append(node.value)
        
    def visit_Integer(self, node: cst.FunctionDef) -> None:
        self.ints.append(node.value)
        
    def visit_Float(self, node: cst.FunctionDef) -> None:
        self.floats.append(node.value)

In [ ]:
file_cst = cst.parse_module(open(file_path, 'r').read())
visitor = FunctionCollector()
wrapper = cst.metadata.MetadataWrapper(file_cst)
jj = wrapper.visit(visitor)

In [ ]:
b = pd.DataFrame(visitor.names, columns=['assigned', 'data', 'line'])
b['assignee'] = b['data'].apply(lambda x: x['names'])
b

In [ ]:
var_names = set(list(b['assignee'].explode().dropna().unique()) + list(b['assigned'].unique()))
var_names

In [ ]:
dg = nx.DiGraph()
for name in var_names:
    dg.add_node(name)
for index, a_series in b.iterrows():
    if a_series['assignee']:
        for assignee in a_series['assignee']:
            dg.add_edge(assignee, a_series['assigned'])
nx.draw_networkx(dg)

In [ ]:
var_name = 'll'
ancestors = nx.ancestors(dg, var_name)
c = b[b['assigned'].isin(ancestors.union(set([var_name])))]
c

In [ ]:
print("".join(np.array(open(file_path, 'r').readlines())[c['line'].values- 1]))

In [ ]:
c['line'].values- 1

In [ ]:
# file_cst

In [ ]:
class FunctionCollector(cst.CSTVisitor):
    METADATA_DEPENDENCIES = (cst.metadata.PositionProvider,)
    def __init__(self):
        self._names: List[str] = []
        self.names: List[str] = []
    
    def visit_Assign(self, node: cst.FunctionDef) -> None:
        pos = self.get_metadata(cst.metadata.PositionProvider, node).start
        collector = ValueCollector()
        node.value.visit(collector)
        value_dict = {'names': collector.names, 'ints': collector.ints, 'floats': collector.floats}
        self.names.append((node.targets[0].target.value, value_dict, pos.line))

In [9]:
print(file_content)

import alkh
alkh.analyze()


class A:
    def __init__(self):
        b = 8
        pass

    def run(self):
        a = 5
        b = a + 7 + 5.0
        ll = a + 6.4
        c = a + b + 3
        d = b + c


class B:
    def __init__(self):
        b = 8
        pass

    def run(self):
        a = 5
        b = a + 7 + 5.0
        ll = a + 6.4
        c = a + b + 3
        d = b + c



In [10]:
wrapper = cst.metadata.MetadataWrapper(cst.parse_module(file_content))
scopes = set(wrapper.resolve(cst.metadata.ScopeProvider).values())
for scope in scopes:
    print(scope)

In [16]:
ranges = wrapper.resolve(cst.metadata.PositionProvider)
file_number_of_lines = len(file_lines)

In [17]:
# go over scopes, get function scopes, for each function scope get node and find call gpaph

In [18]:
a = pd.Series(list(scopes)).to_frame('scope')
a

,scope
0,<libcst.metadata.scope_provider.GlobalScope ob...
1,<libcst.metadata.scope_provider.ClassScope obj...
2,<libcst.metadata.scope_provider.FunctionScope ...
3,<libcst.metadata.scope_provider.ClassScope obj...
4,<libcst.metadata.scope_provider.FunctionScope ...
5,<libcst.metadata.scope_provider.FunctionScope ...
6,<libcst.metadata.scope_provider.FunctionScope ...


In [19]:
def get_range(scope, file_number_of_lines, ranges):
    if isinstance(scope, cst.metadata.scope_provider.GlobalScope):
        start_line_number = 1
        end_line_number = file_number_of_lines
        scope_name = 'global'
    else:
        start_line_number = ranges[scope.node].start.line
        end_line_number = ranges[scope.node].end.line
        scope_name = scope.name
    scope_length = end_line_number - start_line_number + 1
    values = [scope, start_line_number, end_line_number, scope_length, scope_name]
    names = ["scope", "start_line_number", "end_line_number", "length", "name"]
    output_series = pd.Series(values, index=names)
    return output_series

In [20]:
b = a["scope"].apply(get_range, args=(file_number_of_lines, ranges))
b

,scope,start_line_number,end_line_number,length,name
0,<libcst.metadata.scope_provider.GlobalScope ob...,1,28,28,global
1,<libcst.metadata.scope_provider.ClassScope obj...,5,15,11,A
2,<libcst.metadata.scope_provider.FunctionScope ...,19,21,3,__init__
3,<libcst.metadata.scope_provider.ClassScope obj...,18,28,11,B
4,<libcst.metadata.scope_provider.FunctionScope ...,10,15,6,run
5,<libcst.metadata.scope_provider.FunctionScope ...,6,8,3,__init__
6,<libcst.metadata.scope_provider.FunctionScope ...,23,28,6,run


In [21]:
line_number = 26

In [22]:
c = b.query(f"start_line_number <= {line_number} and end_line_number >= {line_number}").sort_values("length")
c

,scope,start_line_number,end_line_number,length,name
6,<libcst.metadata.scope_provider.FunctionScope ...,23,28,6,run
3,<libcst.metadata.scope_provider.ClassScope obj...,18,28,11,B
0,<libcst.metadata.scope_provider.GlobalScope ob...,1,28,28,global


In [23]:
d = c.iloc[0]
d

scope                <libcst.metadata.scope_provider.FunctionScope ...
start_line_number                                                   23
end_line_number                                                     28
length                                                               6
name                                                               run
Name: 6, dtype: object

In [44]:
def _get_call_graph_with_df(cst_node) -> (nx.DiGraph, pd.DataFrame):    
    visitor = FunctionCollector()
    cst_node.visit(visitor)
    call_df = pd.DataFrame(visitor.get_info(), columns=['assigned', 'data', 'line'])
    call_df['assigner'] = call_df['data'].apply(lambda x: x['names'])
    di_graph = _create_di_graph_from_call_df(call_df)
    return di_graph, call_df

In [45]:
class FunctionCollector(cst.CSTVisitor):
    METADATA_DEPENDENCIES = (cst.metadata.PositionProvider,)

    def __init__(self):
        super().__init__()
        self._assign_info: List[Tuple] = []

    def get_info(self):
        return self._assign_info

    def visit_Assign(self, node: cst.FunctionDef) -> None:
        pos = ranges[node].start
        collector = ValueCollector()
        node.value.visit(collector)
        value_dict = {'names': collector.names, 'ints': collector.ints, 'floats': collector.floats}
        self._assign_info.append((node.targets[0].target.value, value_dict, pos.line))


class ValueCollector(cst.CSTVisitor):
    def __init__(self):
        super().__init__()
        self.names: List[str] = []
        self.ints: List[str] = []
        self.floats: List[str] = []

    def visit_Name(self, node: cst.FunctionDef) -> None:
        self.names.append(node.value)

    def visit_Integer(self, node: cst.FunctionDef) -> None:
        self.ints.append(node.value)

    def visit_Float(self, node: cst.FunctionDef) -> None:
        self.floats.append(node.value)

In [46]:
def _get_all_variables_names(call_df):
    return set(list(call_df['assigner'].explode().dropna().unique()) + list(call_df['assigned'].unique()))

def _create_di_graph_from_call_df(call_df):
    var_names = _get_all_variables_names(call_df)
    di_graph = nx.DiGraph()
    for name in var_names:
        di_graph.add_node(name)
    for index, a_series in call_df.iterrows():
        if a_series['assigner']:
            for assignee in a_series['assigner']:
                di_graph.add_edge(assignee, a_series['assigned'])
    return di_graph

In [47]:
_get_call_graph_with_df(d["scope"].node)

(<networkx.classes.digraph.DiGraph at 0x7f0298b8b0d0>,
   assigned                                               data  line assigner
 0        a         {'names': [], 'ints': ['5'], 'floats': []}    24       []
 1        b  {'names': ['a'], 'ints': ['7'], 'floats': ['5....    25      [a]
 2       ll    {'names': ['a'], 'ints': [], 'floats': ['6.4']}    26      [a]
 3        c  {'names': ['a', 'b'], 'ints': ['3'], 'floats':...    27   [a, b]
 4        d    {'names': ['b', 'c'], 'ints': [], 'floats': []}    28   [b, c])